In [31]:
%matplotlib inline

import random
import pickle
import pandas as pd
import numpy as np
import os
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

from pandas import concat
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Embedding, Masking
from keras.preprocessing.sequence import pad_sequences

dataPath = '../PedestrianData/IntentionData'
finalPath = '../PedestrianData/final.pickle'


In [2]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [44]:
files = os.listdir(dataPath)
final = []
for i in range(len(files)):
    file = dataPath + "/" + files[i]
    
    # Reading trial csv to headers
    df = pd.read_csv(file, sep=",", header=None, 
                 names=["Time", 
                        "Pos_x", "Pos_y", "Pos_z", 
                        "Gaz_x", "Gaz_y", "Gaz_z", 
                        "Vel_x", "Vel_z",
                        "Acc_x", "Acc_z",
                        "Vel_Mag", "Acc_Mag",
                        "Vel_Ang", "Acc_Ang", "Gaz_Ang",
                        "Vel_Bin", "Acc_Bin", "Gaz_Bin", "Chg_Bin"          
                       ])
    
    # Transforming data to shifted data (for t+1, etc)
    data = series_to_supervised(df, 1, 2, False)
    
    # Sequence includes all of the features we want
    sequence = data[[
        'var2(t+1)', 'var4(t+1)', # Coordinate at t+1
        'var2(t)', 'var4(t)', # Position X and Z at t
        'var5(t)', 'var6(t)', 'var7(t)', # Gaze X Y Z
        'var8(t)', 'var9(t)', # Velocity X and Z
        'var10(t)', 'var11(t)', # Acceleration X and Z
        'var12(t)', 'var13(t)', # Velocity and Acceleration Magnitudes
        'var17(t)', 'var18(t)', 'var19(t)' # Vel, Acc, Gaz bins
    ]]
    
    # Replace NaNs with 0
    sequence.fillna(0, inplace=True)
    
    final.append(sequence)

C:\Users\rafva\Anaconda3\lib\site-packages\pandas\core\frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


KeyboardInterrupt: 

In [32]:
final_data = pickle.load(open(finalPath,"rb"))
final_data[1].head()

,var2(t+1),var4(t+1),var2(t),var4(t),var5(t),var6(t),var7(t),var8(t),var9(t),var10(t),var11(t),var12(t),var13(t),var17(t),var18(t),var19(t)
0,289.9,653.2,290.0,653.1,1.0,-0.2,-0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,3
1,289.9,653.2,289.9,653.2,1.0,-0.2,-0.2,-0.905756,0.905756,-8.203941,8.203941,1.280933,11.602124,8.0,8.0,3
2,289.8,653.2,289.9,653.2,1.0,-0.2,-0.2,0.000000,0.000000,8.074304,-8.074304,0.000000,11.418791,0.0,4.0,3
3,289.8,653.3,289.8,653.2,0.9,-0.2,-0.3,-0.908670,0.000000,-8.256820,0.000000,0.908670,8.256820,7.0,7.0,3
4,289.8,653.3,289.8,653.3,0.9,-0.2,-0.3,0.000000,0.997460,9.063628,9.949274,0.997460,13.458730,1.0,1.0,3


In [33]:
final = []
for df in final_data:
    final.append(np.array(df))
final = np.array(final)

# Final is an array of crossing trials (sequences)
print("First trial:")
print(final[0])
print("Final shape: ", final.shape)

# Each trial is an array of timesteps
print("\nFirst row of trial:")
print(final[0][0])
print("Trial1 shape: ", final[0].shape)
print("Trial2 shape: ", final[1].shape)
print("Trial100 shape: ", final[101].shape)

First trial:
[[290.  654.  290.  ...   0.    0.    6. ]
 [289.9 654.  290.  ...   0.    0.    6. ]
 [289.9 654.  289.9 ...   7.    7.    6. ]
 ...
 [289.5 649.3 289.5 ...   5.    1.    5. ]
 [289.6 649.3 289.5 ...   0.    1.    5. ]
 [  0.    0.  289.6 ...   3.    3.    5. ]]
Final shape:  (2333,)

First row of trial:
[ 2.90e+02  6.54e+02  2.90e+02  6.54e+02 -8.00e-01 -2.00e-01 -6.00e-01
  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
  0.00e+00  6.00e+00]
Trial1 shape:  (232, 16)
Trial2 shape:  (177, 16)
Trial100 shape:  (125, 16)


In [57]:
from keras.preprocessing.sequence import pad_sequences

# pad sequences
padded = pad_sequences(final, dtype='float64', padding='post', value = -0.01)
print("Before padding: ", final.shape, final.dtype)
print("After padding: ", padded.shape, padded.dtype)

X = []
y = []
for seq in padded:
    X.append(seq[:, 3:])
    y.append(seq[:, 1:3])
X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print("\nX shape: ", X.shape)
print("y shape: ", y.shape)
print("X[0] shape: ", X[0].shape)
print("y[0] shape: ", y[0].shape)
print("\nX_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_train[0] shape: ", X_train[0].shape)
print("y_train[0] shape: ", y_train[0].shape)

n_steps_in, n_features, n_steps_out = X.shape[1], X.shape[2], y.shape[2]
print(n_steps, n_features, n_outputs)

Before padding:  (2333,) object
After padding:  (2333, 1031, 16) float64

X shape:  (2333, 1031, 13)
y shape:  (2333, 1031, 2)
X[0] shape:  (1031, 13)
y[0] shape:  (1031, 2)

X_train shape:  (1866, 1031, 13)
y_train shape:  (1866, 1031, 2)
X_train[0] shape:  (1031, 13)
y_train[0] shape:  (1031, 2)
1031 13 2


In [ ]:
y = y.reshape(y_train.shape[0], y_train.shape[1])

In [44]:
model = Sequential()
model.add(Masking(mask_value=-0.01, input_shape=(n_steps_in, n_features)))
model.add(LSTM(2, activation='relu', return_sequences=True))
model.add(LSTM(2, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

# fit network
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

ValueError: Error when checking target: expected dense_15 to have 2 dimensions, but got array with shape (1866, 1031, 2)

In [ ]:
def seq_generator():
    while True:
        for arr in final:
            X_train = np.array(arr[:, 3:])
            y_train = np.array(arr[:, 1:3]) 
            X_train = X_train.reshape((X_train.shape[0]), 1, X_train.shape[1])
            y_train = y_train.reshape((y_train.shape[0]), 1, y_train.shape[1])
            yield X_train, y_train

model.fit_generator(seq_generator(), steps_per_epoch=30, epochs=50, verbose=1)

### Using arash's data

In [64]:
arash_data = pickle.load(open('C:/Users/WorldViz/PedestrianData/cleaneddata\seqdata',"rb"))
X = arash_data[0]
Z = arash_data[1]
Y = arash_data[5]

X = pad_sequences(X,dtype='float32',value=-0.01)
Z = pad_sequences(Z,dtype='float32',value=-0.01)
Y = pad_sequences(Y,dtype='float32',value=-0.01)

inputseq=[]
for i in range(X.shape[0]):
    mrg_input=np.transpose(np.vstack((X[i],Z[i])))
    inputseq.append(mrg_input)

inputseq=np.array(inputseq)

print(inputseq.shape)

# seperate validation data and test set
tst=0.2                 #% if test data
#val=0.2                 #% if valid data
tstsize = int(np.floor(len(X) * tst))
#valsize = int(np.floor(len(X) * val))

tstlabel = random.sample(range(0, (inputseq.shape[0]-1)), len(range(tstsize)))
inputseqTEST=inputseq[tstlabel]
ytest=Y[tstlabel]

trnlabel=[i for i in range(inputseq.shape[0]) if i not in tstlabel]
inputseqtrain = inputseq[trnlabel]          #Excluding test set
ytrain = Y[trnlabel]

n_features = inputseq.shape[2]
n_steps_in=inputseq.shape[1]
n_steps_out=ytrain.shape[1]

model = Sequential()
model.add(Masking(mask_value=-0.01, input_shape=(n_steps_in, n_features)))
model.add(LSTM(2, activation='relu', return_sequences=True))
model.add(LSTM(2, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

# fit network
model.fit(inputseqtrain, ytrain, epochs=50, batch_size=1, verbose=1)

(3304, 83, 2)


KeyboardInterrupt: 

In [93]:
arash_data = pickle.load(open('C:/Users/WorldViz/PedestrianData/cleaneddata\seqdata',"rb"))
X = arash_data[0]
Z = arash_data[1]
Y = arash_data[5]

X.shape, Z.shape, Y.shape

((3304,), (3304,), (3304,))

In [94]:
X = pad_sequences(X,dtype='float32',value=-0.01)
Z = pad_sequences(Z,dtype='float32',value=-0.01)
Y = pad_sequences(Y,dtype='float32',value=-0.01)

X.shape, Z.shape, Y.shape

((3304, 83), (3304, 83), (3304, 147))